In [1]:
# ---
# title: ReaxKit fort.79 Parser and Analyzer
# description: Demonstration of parsing and analyzing fort.79 files using ReaxKit.
# ---

# ## Step 0: Imports
# - `Fort79Handler` to parse the fort.79 file
# - `pandas` for handling the resulting DataFrame
# - `Path` for safe file path management

from pathlib import Path
import pandas as pd
from reaxkit.io.fort79_handler import Fort79Handler  # adjust import if needed


In [2]:
# ## Step 1: Parse fort.79 File
# The fort.79 file often contains per-frame simulation data (e.g., dipoles, stress tensors, or electric field components).
# Here we use Fort79Handler to parse it and extract both metadata and numerical data.

path = Path("fort.79")  # or provide full path
handler = Fort79Handler(path)

# --- Try the correct call based on implementation ---
# Some ReaxKit handlers return a (df, meta) tuple from _parse(),
# while others populate .df or .data attributes instead.

result = handler._parse()  # Internal parse method (returns or populates data)

if result is None:
    # If nothing is returned, try the explicit public parse() if available
    handler.parse()
    if hasattr(handler, "df"):
        df = handler.df
        meta = getattr(handler, "meta", {})
    else:
        raise RuntimeError("fort79_handler produced no DataFrame; check attribute names.")
else:
    df, meta = result

print("\n[Done] fort.79 parsing complete")
print(f"Records parsed: {meta.get('n_records', len(df))}")
print(f"Columns: {list(df.columns)}\n")
display(df.head())

# Save parsed data to CSV for later analysis
df.to_csv("fort.79.csv", index=False)
print("Saved parsed data → fort.79.csv")



[Done] fort.79 parsing complete
Records parsed: 2002
Columns: ['identifier', 'value1', 'value2', 'value3', 'diff1', 'diff2', 'diff3', 'a', 'b', 'c', 'parabol_min', 'parabol_min_diff', 'value4', 'diff4']



,identifier,value1,value2,value3,diff1,diff2,diff3,a,b,c,parabol_min,parabol_min_diff,value4,diff4
0,4 13 1,0.33290,0.33390,0.3334,16975.52522,17023.16057,16994.88318,1.783885e+07,-1.184731e+07,1.984001e+06,0.332065,16963.08245,0.332065,16963.08245
1,4 13 2,1.82170,1.82270,1.8222,16942.43721,16948.32666,16944.96097,1.683848e+06,-6.130728e+06,5.597285e+06,1.820451,16939.81083,1.820451,16939.81083
2,4 13 3,10.13865,10.15115,10.1449,16930.07205,16940.91897,16937.16370,-4.270578e+04,8.673596e+05,-4.387103e+06,10.155060,16941.57138,10.138650,16930.07167
3,4 13 4,1.43830,1.43930,1.4388,16925.50522,16935.05646,16930.09203,7.552263e+05,-2.163688e+06,1.566616e+06,1.432477,16899.89336,1.432477,16899.89336
4,5 71 1,89.75000,90.25000,90.0000,16885.00469,16867.63206,16882.97753,-1.065465e+02,1.914362e+04,-8.430164e+05,89.836948,16885.80980,90.000000,16882.97715


Saved parsed data → fort.79.csv


In [6]:
# ## Step 2: Analyze Differences using fort79_analyzer
# The `diff_ratios()` function calculates ratios or differences between quantities
# across frames — useful for detecting convergence, stability, or evolution.

from reaxkit.analysis.fort79_analyzer import diff_sensitivities

# Create a new handler instance (optional if `handler` already exists)
handler = Fort79Handler("fort.79")
df, _ = handler._parse()

# Compute ratios or differences
ratios = diff_sensitivities(handler)
print("\n📊 diff_sensitivities() result (head):")
display(ratios.head())

# Export to CSV for documentation or further plotting
ratios.to_csv("fort.79_diff.csv", index=False)
print("Saved difference sensitivities → fort.79_diff.csv")



📊 diff_sensitivities() result (head):


,identider,sensitivity1/3,sensitivity2/3,sensitivity4/3,min_sensitivity,max_sensitivity
0,4 13 1,0.998861,1.001664,0.998129,0.998129,1.001664
1,4 13 2,0.999851,1.000199,0.999696,0.999696,1.000199
2,4 13 3,0.999581,1.000222,0.999581,0.999581,1.000222
3,4 13 4,0.999729,1.000293,0.998216,0.998216,1.000293
4,5 71 1,1.000120,0.999091,1.000000,0.999091,1.000120


Saved difference sensitivities → fort.79_diff.csv
